# IMPORT

In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import json
import pandas as pd
import numpy as np
import requests
import re
import os
import datetime
from datetime import date, timedelta
from datetime import datetime
import glob
import time 

In [3]:
pd.set_option('display.max_columns', None)

# AUTHENTIFICATION
* guide for authentification with Spotify API: https://developer.spotify.com/documentation/general/guides/authorization-guide/
* calling personal credentials from environment https://able.bio/rhett/how-to-set-and-get-environment-variables-in-python--274rgt5
* open in terminal: touch ~/.bash_profile; open ~/.bash_profile
* implement: source ~/.bash_profile
* get new token https://developer.spotify.com/console/put-playlist/


#### Create and refresh token
run this to  refresh the token. Token runs out every 3600 seconds.

In [4]:
my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')

my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')
sp=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret))


#token to pull songs of an individual usersp_pers=spotipy.Spotify(cache_token_personal)
cache_token_personal= '' 
sp1=spotipy.Spotify(cache_token_personal)


# CRAWLING FOR CHARTS
HERE WE DOWNLOAD CSVs FORM www.spotifycharts.com

### Function
downloads CSVs from spotifycharts.com given the countries of choice and the time interval set below

* Note: this depends on the availability of the CSVs on spoitifycharts.com . Error may occure if we are trying to pull data for a day spotify does not provide charts data for. Slightly adapting frequency or start and end date might shift you around thos "data holes"

In [5]:
#DOWNLOAD LOOP FOR COUNTRY AND FOR TIME PERIOD
# prepare a folder named 'data' with two subfolders named 'viral' and 'regional'
def bulk_charts_download(country_charts, date_list):
    for country in country_charts:
        for date in date_list:
            url= f'https://spotifycharts.com/{what_charts}/{country}/daily/{date}/download'
            r = requests.get(url, allow_redirects=True)
            open(f'data/{what_charts}/{date}-{country}-{what_charts}.csv', 'wb').write(r.content)

### SET PARAMETERS FOR CRAWLING

#### Set Start and End Date

In [6]:
startdate = date(2018, 1, 1)   # start date
enddate = date(2021,1,30)   # end date
delta = enddate - startdate       # as timedelta

date_list=[]

for i in range(delta.days+1):
    day = startdate + timedelta(days=i)
    date_list.append(day.strftime("%Y-%m-%d"))

#### Chose frequency
between start date and end date how often do we pull charts data

In [7]:
date_list=date_list[0::30] #every 30 days

#### Country Codes

In [8]:
#enter list of country codes
country_charts=['us','br','jp','de','ca','gb','tr','is','mx','za','au','ar','eg','es','it','ma','nz','id','ru','ua','in','']
#HELP: LIST OF COUNTRY CHARTS
print(sp.country_codes)

['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']


#### Which charts are we crawling
Options: viral charts or regional top200

In [9]:
# VIRAL CHART sample URL https://spotifycharts.com/viral/us/daily/2020-03-13/download
# thats how i want filename to be safed regional-us-weekly-2020-03-13--2020-03-20.csv
# enter as string 'viral' for viral top 50 or 'regional' for local top 200
what_charts='viral'

#### Trigger download CSVs to folder: Calls function with parameters set above

In [10]:
bulk_charts_download(country_charts, date_list)

# CREATE DATAFRAMES

#### create dataframe from csvs
* intermediate step before adding music features

#### FUNCTION CSVs to Data Frames
* additionally extracts data on country, date and song IDs form the original URLs


In [13]:
def csv_to_dataframe(what_charts):
    path = f'data/{what_charts}/'                     # use your path
    all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

    df = [] # pd.concat takes a list of dataframes as an agrument
    for csv in all_files:
        
        # if condition since csv of top200 look differently than csvs from viral charts.
        # top 200 sharts do have a disclaimer not in the first line that messes with our columns upon import
        # we need to igonre it
        if what_charts=='regional': 
            frame = pd.read_csv(csv,skiprows=[0],error_bad_lines=False, warn_bad_lines=True)
            frame=frame.drop('Streams',axis=1)
        #this is the read in for viral charts csvs
        else:
            frame = pd.read_csv(csv,skiprows=[],error_bad_lines=False, warn_bad_lines=True)
        
        frame['filename'] = os.path.basename(csv)
        df.append(frame)
        
    #concat list of dataframes into 1
    df=pd.concat(df, ignore_index=True) 
    
    #finetuning columns with regex
    df['ID']= re.findall(r'\d\w+',str(list(df['URL']))) #extract ID from URL
    df['date']=re.findall(r'\d\d\d\d-\d\d-\d\d', str(list(df['filename'])))
    df['date']=pd.to_datetime(df.date, format="%Y-%m-%d", yearfirst=True)
    df['country']=re.findall(r'-[a-z][a-z]-', str(list(df['filename'])))
    df['country']=re.findall(r'[a-z][a-z]',str(list(df['country'])))
    
    # indicates if song is from viral charts or top 200 charts/ 'regional' indicates its part of  regional top200 charts hence we label it 'top200'
    df['group']=np.where(what_charts=='regional','top200',what_charts) 
    return df

In [14]:
viral_df_basic=csv_to_dataframe('viral')#pass in bracket if 'viral' or 'regional'
len(viral_df_basic)

34250

# SUBSETTING VIRAL CHARTS
* to make the project feasible for my processing means I need to subset
* my machine is able to make approx 6300 calls to the api per hour
* subsetting to the viral top 20 of 22 countries between 01/01/2017 and 31/12/2020 at 12 sample dates
* ending up with almost 5872 ids to represent 13827 entries

In [15]:
# unique song id: plenty of songs are in teh charts multiple time for different countries and at different dates
# lets see how many IDs are unique since we dont want to pull features for the same song multiple times
unique_ids=list(viral_df_basic.ID.unique())
len(unique_ids)

14108

In [16]:
# lets return all unique IDs of songs that are at one point place in the top20 viral charts in any country at any time

unique_ids_of_top20viral=set(viral_df_basic['ID'][viral_df_basic['ID'].isin (unique_ids) & viral_df_basic['Position'].isin(list(range(1,21)))])
len(unique_ids_of_top20viral)

6008

In [17]:
# using the top 20 of each countries viral charts at all sample points results in 5872 unique IDs
# thats below my 6300 ongs threashhold

In [18]:
#
viral_df_basic_top20=viral_df_basic[viral_df_basic['Position'].isin(list(range(1,21)))]
viral_df_basic_top20.shape 

(13700, 9)

In [19]:
len(viral_df_basic_top20.ID.unique())

6008

# INSPECT DATAFRAME

In [20]:
viral_df_basic_top20.head()

,Position,Track Name,Artist,URL,filename,ID,date,country,group
0,1,Sucker,Jonas Brothers,https://open.spotify.com/track/4y3OI86AEP6PQoD...,2019-03-27-in-viral.csv,4y3OI86AEP6PQoDE6olYhO,2019-03-27,in,viral
1,2,Undrunk,FLETCHER,https://open.spotify.com/track/5SHhPFh68OhUmuR...,2019-03-27-in-viral.csv,5SHhPFh68OhUmuRPymKX9d,2019-03-27,in,viral
2,3,Don't Call Me Up,Mabel,https://open.spotify.com/track/5WHTFyqSii0lmT9...,2019-03-27-in-viral.csv,5WHTFyqSii0lmT9R21abT8,2019-03-27,in,viral
3,4,Moonlight,Gaullin,https://open.spotify.com/track/0A5gdlrpAuQqZ2i...,2019-03-27-in-viral.csv,0A5gdlrpAuQqZ2iFgnqBFW,2019-03-27,in,viral
4,5,i'm so tired...,Lauv,https://open.spotify.com/track/7COXchtUOMd6uIT...,2019-03-27-in-viral.csv,7COXchtUOMd6uIT6HvmRaI,2019-03-27,in,viral


In [21]:
#are their songs that are in the charts of muliple countries ?
pd.pivot_table(viral_df_basic_top20,index=["Track Name",'Artist'], values=['country','ID'], aggfunc={'country':np.sum,'ID':'count'}).sort_values('ID',ascending=False).head(5)

,,ID,country
Track Name,Artist,,
Dance Monkey,Tones And I,89,deesdemxnzmxbresidnzegmamaesegbrbrmxidtraregnz...
death bed (coffee for your head),Powfu,61,jpitzazaitarisusaugbcausgbruaritjpzacaauusisgb...
Roses - Imanbek Remix,SAINt JHN,50,itzacaitarisusaugbcaisaucausgbruitgbcaaunzisgb...
ily (i love you baby) (feat. Emilee),Surf Mesa,50,itzacaauitisarusgbisusgbgbusjpzacaauauuaitcais...
Sweet but Psycho,Ava Max,50,zaarititaritzaauususgbaugbzadejpjpgbnzauismaca...


In [22]:
#anything empty?
viral_df_basic_top20.isnull().sum()

Position       0
Track Name    12
Artist        13
URL            0
filename       0
ID             0
date           0
country        0
group          0
dtype: int64

In [23]:
#NAN values
viral_df_basic_top20[viral_df_basic_top20['Artist'].isnull()]

,Position,Track Name,Artist,URL,filename,ID,date,country,group
2700,1,NaN,NaN,https://open.spotify.com/track/47rbjDud83d3aqv...,2019-04-26-in-viral.csv,47rbjDud83d3aqvbcTssei,2019-04-26,in,viral
3652,3,NaN,NaN,https://open.spotify.com/track/74kCf4Yd2jr91yi...,2021-01-01-br-viral.csv,74kCf4Yd2jr91yinngEt0P,2021-01-01,br,viral
7113,14,NaN,NaN,https://open.spotify.com/track/4LAgGxPsaI4HeOJ...,2019-11-22-mx-viral.csv,4LAgGxPsaI4HeOJStF8tgV,2019-11-22,mx,viral
8558,9,NaN,NaN,https://open.spotify.com/track/2IHTNejiUvoSTsF...,2020-10-17-mx-viral.csv,2IHTNejiUvoSTsFpovaESS,2020-10-17,mx,viral
8564,15,NaN,NaN,https://open.spotify.com/track/4xioHVjM8WJmLEV...,2020-10-17-mx-viral.csv,4xioHVjM8WJmLEVd5DD8BQ,2020-10-17,mx,viral
8566,17,NaN,NaN,https://open.spotify.com/track/6eiVcW005z9GU9D...,2020-10-17-mx-viral.csv,6eiVcW005z9GU9DpoEdlqh,2020-10-17,mx,viral
12561,12,NaN,NaN,https://open.spotify.com/track/2AgPxySIeAnXDHZ...,2019-06-25-es-viral.csv,2AgPxySIeAnXDHZvAQHg0R,2019-06-25,es,viral
15316,17,NaN,NaN,https://open.spotify.com/track/47rbjDud83d3aqv...,2019-05-26-in-viral.csv,47rbjDud83d3aqvbcTssei,2019-05-26,in,viral
15509,10,NO GOOD,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,2020-07-19-jp-viral.csv,4Qnz8tARYhUtDNePNsLx2C,2020-07-19,jp,viral
19060,11,NaN,NaN,https://open.spotify.com/track/461JKAn7H6Sbx0q...,2020-12-16-in-viral.csv,461JKAn7H6Sbx0ql9IvRUG,2020-12-16,in,viral


In [24]:
# how many songs per country 
viral_df_basic_top20['country'].value_counts()


es    760
nz    760
it    760
tr    760
us    760
id    760
au    760
gb    760
mx    760
ca    760
jp    760
br    740
ar    720
de    700
za    700
is    680
eg    540
ma    540
in    440
ua    140
ru    140
Name: country, dtype: int64

# CALL API FOR MUSIC FEATURES

### FUNCTION: GET MUSIC FEATURES FROM API

In [25]:
def getTrackFeatures(df):
    feature_names=['ID','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'key','instrumentalness', 'liveness', 'loudness', 'mode','speechiness', 'valence', 'tempo','loud_start']
    features_df=pd.DataFrame(columns=feature_names)
    
    causing_error=[]
    
    for id in set(df['ID']):#only loop through the unique IDs to enhance performance
        try:
            print(id)
            meta = sp.track(id)
            features = sp.audio_features(id)
            analysis= sp.audio_analysis(id) #does not seem to be available at all songs

            #collect in dictionary
            features_dict = {}   

        
            # meta
            features_dict['ID']=meta['id']
            features_dict['duration_ms'] = meta['duration_ms']
            features_dict['popularity'] = meta['popularity']
            # features
            features_dict['acousticness'] = features[0]['acousticness']
            features_dict['danceability'] = features[0]['danceability']
            features_dict['energy'] = features[0]['energy']
            features_dict['key'] = features[0]['key']
            features_dict['instrumentalness'] = features[0]['instrumentalness']
            features_dict['liveness'] = features[0]['liveness']
            features_dict['loudness'] = features[0]['loudness']
            features_dict['mode'] = features[0]['mode']
            features_dict['speechiness'] = features[0]['speechiness']
            features_dict['valence']=features[0]['valence']
            features_dict['tempo'] = features[0]['tempo']
            #analysis
            features_dict['loud_start']=analysis['sections'][0]['loudness']#loundness at the start
        
        #error handling
        except:
            causing_error.append(id)
            print('causing error: ',causing_error)
            
        #wrap up of all gathered features
        track_df = pd.DataFrame(features_dict,index=[0],columns=features_dict.keys())
        features_df = pd.concat([features_df, track_df], axis=0, ignore_index = True)#concatinating along axis 0
    return features_df
    #lst_columns=['Position','Track Name','Artist','URL','filename','ID','date','country','group','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'instrumentalness', 'liveness', 'loudness','mode', 'speechiness', 'valence', 'tempo','loud_start']
    #df_with_features=pd.concat([df,features_df],axis=1,ignore_index = True)
    #df_with_features.columns=[list(df.columns)+list(features_df.columns)]#creates new joined list of columns
    #return df_with_features

#### call API on IDs returning a dataframe with features of each ID

In [26]:
#dataframe of features
viral_features_df=getTrackFeatures(viral_df_basic_top20) #get features for each song


5PWzxlxfeowsA6QaNUQhVZ
2gXZp1zDDA5bnPsLwrSMRj
6A9QEWYXpXLMeJLRjGsIJ1
5kQre7o8ky5DXeEPcyaCR1
6FyRXC8tJUh863JCkyWqtk
30iUCYpZbayLhsY2jrGSBi
60duJnisZ0yYzTieIFiZsN
3T5TvGiNkXKKCgQWKhIfKU
2BuBXn15gj4MNYZydyWgrk
4msighkG4nKOw60aoN7lV7
3TfLZodlQ1TzNpdrc4KXR5
5jS8H3e5ehSHgEfAYoxF2s
5YmeqbDcLOEYwRJvctsWYj
3aUTkuo4ObA6QPtZSu5NBq
0oZRDruCnbV2W9Z6VZaJ1v
5p1ex0pXv6jSPJ6QbumQpD
4ESPuOrSsRcBISzcMlzmY5
5C4tPxg8bWj8E80BZTH7ey
7mVel2ssIsIo3vdsggUvjE
4Ft4Tj0l8kzfKBv8vGuR4t
6azjD02gDO2ZarIZMs3eRk
4l3Gwyi6qjuQOMATThnaCu
7uHRKSQ9WIlBBuBY7BHcbq
37I2kAE7QRxd9WCax1Z9dE
66yik17dX3eiuS5PetLJ0X
2Xoc4gqPbBa3QHKpBLJ5l8
4xc6RrWzEOdprPqYn9Cyen
7arXjVN1RwhEL7pJBAOv5F
31EKU3oevb6I4up2oSYyfS
5r7uA7AxZi1tSBr8Kj4svH
6jgoHNL1ClNlt3XuW1yCJA
2r77vqSZkOktTSgmAOoHsV
22sESxzbY1O1E19xjOTaYf
79qxwHypONUt3AFq0WPpT9
6U1bHEDQPkLMtoBMuxmAmd
3wfldFmVsFWAZZrqw8FxMp
2k6RWRsm2yYD8lOJjer5qa
6cqIxECF2y28W51gAdvzmU
2j1r3ubqqxuoTFaPVnTV1j
6uFYKnQ0jYPAvx2hF1kncC
3id2EVGKc3eFAMn9nFnB0r
7swocJUCUWTCiRUAU9oerC
0Wq6iAraiUpSbBSPmv4gLu
7MzrIZwgEyx

2ptHQBO7TQhEkXOeK9ST59
1ENdcyhqdylaLsPVgB83qq
2G7vVF2QPQtlnQ7EQntij8
21WVC4YuFUgakOEDkTBwLd
017PF4Q3l4DBUiWoXk4OWT
6ele4cqsOo9wDWrRQIjPiw
1cY1no6J4Xp6KCR198gWHQ
1pyRfXYainadEMZXQmmyig
3xStfBqH2vrjoUtzpSdoov
69jegGcdGD7MmBCfM0ledN
73wcFiFeCQurmHDQIcirbm
3nrxsQEntBXdY5JoqA39ES
09ts3GnICqYEU5PkQCpJK3
4qjW4odsAkX0daeqRC6sXL
58QRuYKc1SQHJwFjO7iRY6
0HKsqXx6rZIyA4jiDkbXUE
3ZCTVFBt2Brf31RLEnCkWJ
7piIo15vVy3ULGXSpFs5xc
6ymTp1jOINm0UURUZ2h603
7tS9oGkoSnAl4Nf7UwXar8
28bdcUznUDPjJEujZCABes
5HRHVvNCQpKNno2IAo8esd
2j4nCq2SsdsBMDJjCvlLOw
3k5oLgungD1dSOGLqQdIQw
3jtCPldxNkWPPYyVH7kQyp
0H0MIfeqip6GKmafftx7kC
3JN6cbMhHVyI2UbABfKVhY
02i751eiU2c0lKnvBTWuD8
5UcYibHLpjrqqZKbjEsLR9
35KmOEIUqAGn02t7n4qaIr
2IHTNejiUvoSTsFpovaESS
0M51AEITqiugZHNR5Isi8W
31AuJaUuJV7LI0grZeJM1y
0lEyInit4ezJcYJOBbaVej
1rgnBhdG2JDFTbYkYRZAku
5rZlwNFl01HqLWBQGryKSm
7lT1dCz96jANsLAAADlfIg
0AkF8p56DQ3qdaKbtX4VCD
2h8kIHr1Nb0gOVdRxJEoQK
0FqKrH1lJPwSjFyLu8ndlN
20xxauHsSIf5CkDxcNW2DG
0ZEYRVISCaqz5yamWZWzaA
1h2PcYeuP3DMwrPSxV0b76
3eWLKn5HOZD

2Yer0p7uB2lVBUAtANuuQp
512PRgcIjizP6KFJQLcRDF
1I7FBJDOWgKtqyTGfuzsr8
3OrzQ7xwaLyhEV7M10CqCS
7wk0e9D9bFAFEigOvUX9fB
2rjCLFdX8n9xLoMZgdMMxt
53QiI2eOSXB4GPkPsoMd0I
5nSH1TJNhEoagTneTmvqPG
6hyfLcl4tdjKB4AGARbXET
2Sla5cgNpVoRcckYjaAMKP
04YutdH0j1INpsva3gOipt
4iN8XAk63IUwMPBYKiE0kD
1lTsRXDMWFbsFjQGkbL5TX
07fOPxk9AQ96qoqJkTVFHe
72Yg5qdIqpTnXrNPEewglz
5wdlz4N7s1Xyr4y4RXhZJM
7oQZR3Bq6HMrTdkqcbCJNf
1GOiMxQXZUzjzH0Sxpck0B
1ktxOFiZjy75OmIXvgYnNl
1Td7TGT1XtK2ojUjz1mGUV
5TgS4dcUAU8EEb506d8wAT
2z8R3K0bp0URKH9eL8PszO
5ZwxJQ8vR5cHWerXU8fb3f
5UVa8Qqb9L1dVxl3jKQ5J1
7KJAGp6bvOMsz0qF8kTaLt
43Gxk31GJtnHot0tCpTr6J
51uFW0zfJXSe4OJDS9urmt
4W3vYvW8ivZuKM9UdMnhM9
42grBspOHbj3eWZTXV8TK2
43wzy7JxMEfvCh8ZFeZKYk
0m0Ew7RcwZaBS8psxgE00P
40MlMSv2kkmph6XhYMOovm
70J8t2qwQ2YbSaiUfO3ESs
0lIBuKdkNrf1jVSwfMpVEG
7jIfxFhZSmH3LGud6D5m2w
2ritsV4U3jq2LduJpovZ1A
5XaWWhvsXlqziD2IRCuPgI
3QqKJXiqBkgMM2prOdByeC
6TesHoNPZGdcVjllhAuKJh
7K8krdGeSIvwDGw1WUqJuK
6y8lyV4fUWkyL0B4kMudSQ
0YYNIGKgAk34vZkV2NySie
3UltPYuSY57T9BBAtN8w9L
1MUL6DO2gVi

1HagMz5nWk1HmevNCBOxF3
6wQlT5VQU5eM4NYBIJ6wO5
494sQS9uA5kC2ac41KttTo
0vWUhCPxpJOJR5urYbZypB
2xbt9VePhuPH3xH0ceAuA8
5RKClHNCc5x2BritkOF7tI
1oVxLao17vSSmzGztTdwrP
4NzMOnvSJVNKF7nw5NkXIP
5N1OCw4KTdu1cbcecdSsDM
1IIKrJVP1C9N7iPtG6eOsK
6fpPKJETCAD63HOUI36jhI
224EVTFiLsdQignUFLvDeH
0Rjx8PXGkO8BANQSFNmYfq
1lSKbTaBNDu8D9mTNu3nhy
1yTJg3lyUPmwbnve82twH5
7jeayaJnIrd9GvDJQllS6K
0P2Dxoy0mdzM9NlFQoLZbc
5GOYSK6UMXF3rG6sYUoJ2j
5N4erncE7kuUccm7zEmwzk
69YDupexPCFwKmEVejktZ5
79i617mqpQLwUs3WTokYX9
2exdDhuNjyxtmJIKkbSWbo
48XfojJC87VKLlUotYV0Gm
07Lxxln9lZ8m0hT6EySTCe
6W1VSuiDWxIIiz5YijN7UP
2z7wqrpcBQd3y061E8KypF
172zA5Yn0YzayQWvEJuGAm
0mXkEGn0LyuZTsdnoHvO7f
6C6mUonH6MNdSg5pw7kOc9
1v9OR23wz65Tkw4vaqo5t1
18Gcl5bxdd8bpBhWOj1rCR
3NpRkAnxexTh4TklAZNjag
5x0XcS4mIYnANEk5i7H80v
5jtgmZRPmm6QnJxegPoour
24HgQlk5MZsXEB1ZMBQTdI
6KmlDP61GjKhwH8mldtIYP
4fXGWiVhlOLdhwRDP6pIFG
6fWTVk29yvFEqE5VY6Gslk
6aDo3jkC0zusH8ycwismZJ
4YMfE4r1JWADFw2Fn74XW2
04UzZSWfw8hkK832P27QLq
4mQeW3jdPoseyOlyHbBa4E
5vLEmh5EolySKTvXsyWSOg
248wt1trbhw

4z2JaAVPemYJhFKm7e32RA
1GWsbdHJydpGFAqDDfLJHR
3EPXxR3ImUwfayaurPi3cm
6ocbgoVGwYJhOv1GgI9NsF
3d095101OA0cMGI3iW1AyP
4X8S3fYTRC5LJA4ZIFCbS3
4IrdTItUDGxT4zOnRkKclN
68bvCjofUJVOZcaM0lz0Vc
73F8RdXGiuw93C7cEiPYIR
23TPP1eeElFfvYVznskwCY
3T00mhdOYUuM5yiuPs3xhs
3GTRCT1awGpTwi3ONuT8D3
72gC51qe5TmbkrcCMThAHw
2F7xRkUetyKpe4LzOITHZP
46FGVzlUEtTYfSYamzBKTB
4NzsZHfwEWCaxlyUV95Aai
1mohfLaTJtB2RplHLQVM70
3YNcQUeVOpM3SDmwBeGfMK
3fCyP5h59OZOgfqQbeO1lJ
5KSNUteuNykJDjYiqguIzk
0ophTl8mevsVgtH7V0XV0g
3pZDZloBhUwlKA5O6JzXXa
6EfZhfuSOVC8qnvCGkIQU5
56JyMaElW79S7TDWh1Zw1m
5iFwAOB2TFkPJk8sMlxP8g
4H9nfkoXso9fyLkQExZDjA
5GbVzc6Ex5LYlLJqzRQhuy
2Ut2aIu0jalbdzemOjMV7a
5JIGmdJjGKukNFJzPnWRUQ
3wtgAhO0Jo2MDdpwlJEOSg
3TaGtHDHWQ7TfPpX0eJrtB
0MsZ9Les11qQY8NEBA9Pwd
1iIhGHzzrzqQfuNkFI2qAn
4QIo4oxwzzafcBWkKjDpXY
7fT5flhPP1Hy5kYyO98SU0
4GMO1kuFmcuD0tSPPi1cpo
2QyuXBcV1LJ2rq01KhreMF
5Ws6qds3QAQ5s4tN5P2zRA
1c5ZiOfMbDBHKRNuMfinOb
2wvLtM25eMpHK7hkidqte2
14fysanCY4W2wJlsp8wtl7
22LAwLoDA5b4AaGSkg6bKW
7rTXWBEwokHB8HX6d5daLI
5tnxlzJHU3E

...

4WYRFY0yFa2s1WeX0w84z6
184XtKkzoAl83quIarUAJP
26WpOBpHiIMoBuMEIwtOCH
3zeRiTAkesKi2kkT0qeHEZ
3zefoLCDoPU5PJK6g1EHoF
01vR5Z7AHinp9D2wu2BNQS
2pia5xldH8dTPOzxqn5dvC
0GiwV6v3AgJfdu59tj719Y
48VlmYWmONggBCfvz0I9ck
4J4zx8xGJpuGndrVc06KaT
3eaEZpGIWld5r6kEeN4zWX
5N377bFg76cTyyASoU0nBD
7K9kw58ieqM9vz2E2SeOcN
7EAtJk0HzXQp2gpFeaaLzF
7C3xydn7tJZLCSBcV2ZFYe
22rcaYTa2UgcLCbeO0NoNj
7lhBwSHmExAX1KGPOjAlm1
0Jt6MZYolVvJUF9yS1xeTY
3i5vHOzl2KVCBnfqgsFC9V
2mMorGg2R2NfAbvdrv3Cgn
4HYDUMY0xSpeBr0AMY9cUz
4YJUTdZ0Pgl0ZeNyHYXeLd
2he61MJyMIOj8enfDmLxRI
1snWlbcbgQpJfknoI30DWG
5zOU1O7M2NtWldQcbEr0MD
1ZmUv2GZTHIUXNKd1KUIu2
3sELGQFkm5mIvjM69u3Qhb
21SBUUtRtPK2x2OLmZCKTP
6XvQPMLZdPPV22rY7FEBpI
4MzXwWMhyBbmu6hOcLVD49
2pL1V75Sv9ZUTJuWrRcu0v
2CRbiMK0W14l71SpDKqKZM
0Wv23nhntPTf4xvIcp6dgm
4tooe2eEbvqb0RewwhLQbD
61A9xo1iyPjkPYfy49suAf
5PYQUBXc7NYeI1obMKSJK0
5Dxnfi8E00VyYRjfl8LDJm
4Ybp3UvVXvraRNwOCWnrNq
2j8cgzNl4MzhTWHfobOJDO
1IJH7QjV5V99eJBtKXqGXE
53CJANUxooaqGOtdsBTh7O
7M0fsXo6rpXOKkveYooBob
7HXEu2QMLAbGTccwQpV9Xl
4sa0pXDdQkB

4ekRE8jtEHBH9palfFvtqG
01oZM8lBFS8harAIonArgi
29aGW9bHQVgaarIVAsX4yi
2O866dotlltmzcngBvEa31
7dvNoLkDLGxZA8DNwvJleu
6rTPGpRB9jbTo9FPlmR6J7
1CmP16cGnlRGMcL0S7q50s
43sNw9nvGYGLYnNiwI8oMZ
4UiaP0wBuy9N9voD91OZOg
4Sn5B44sLfQ364FUL98jvN
3VC7KLsQnwLdKHRu7J6XBP
6Oo0ceqCbpwxUPQjl70lNJ
2yuYI5NFhevxa05se7Qht9
5mIOsPuQdXchVY0jB5NO9Q
5g9mbYv5JL9AIa9lKw2EHJ
07Hmp0XsNVgspry3T7K1Xq
21kOVEG3bDCVphKhXL8XmQ
6afz6WNkVxp0q3anZU72bh
52X2jsXZIffZmp5ONxHwQl
3DG7EHsDkC6tj7D0oQNwWK
3NxFTRr0692mbC5hD0nEZD
2LMW0Tce3jhJT2pqPSxL3q
4UJoAdTMnqxQHKWKLLQpvK
1eZqpVKXeioGYIH4gHDU3n
5F6tLUQGd7GtiWID9ZEzev
6vKqPHrfOVJMsmuVNa4aqy
2zAKwUD7TVTt8qTxa9bnQJ
7fvr9JMI9DCuty3882lZ9t
2jLwKWTyxexSxvyDKjXxVk
6NprDUe0yiLIVGgcnFGgtq
2jX5c5RFp0A9E1GDsvGxIa
4QIwkysLEV8vChxapCm30P
1ABJmQDljo3YidXJhexQBP
6XYOGO4lMhj5uIgOosOrnt
4q0q5o5BTp2sl24t8iZ8aN
26Xg0wmftyPdzxCNFPgw2i
17q1nFo1UQvXiQ8TD7aI8H
1Nyh4t2z67Lp7kiTDccWtu
0vmwu4CJzvkx2P8T4yo2lX
7qpQDJDDogmn0DFFydSoMR
6ho0GyrWZN3mhi9zVRW7xi
04R6Y6bBCpuEGd6nuS0uQE
0PKEQjnGn0LmPjozJ3t4MF
4MszwagGVhB

2PfJwLxlzWkZoetIAJICDV
2LYt5nf0m90Bz8fuudLQd1
3qvYObuTA1qbfL9LpeRtLx
0pSq5GJnniAx4YPvfgaWIO
5TVpVYcU47B23onx6llnvv
0osdzA0D159EUmBcrwMcB4
7x9nXsowok1JszkVztI5NI
4Zkjhf26f6Ai7qSMjt1Yue
3zwaujwhco633gIls5LVac
7DRDJ5ZzU7mj7CfRxGYzOY
1bIFQU5702DNqLHVnickWt
0gaGGc6PhaYmjnnY2Wrogi
7hxHWCCAIIxFLCzvDgnQHX
54OqGd3dmug64db0ZtOtF4
73jhZZreWFHhDHInPPor8s
0haX1okP3ZHnhWPzDAUipW
7Lf6EGbqAzddC6dYKpsld5
3tRtlS9YUyYqVgqO3iM4bW
6MJUCumnQsQEKbCy28tbCP
2S93hylQBYLndMnaJuso8S
76KDy62J99FKu6ZsxMJy3j
1BwrZ8tZ3sN7cMqDoQ91hd
2hLNnDcwlBzTr9R1QQJw01
5vnuJ8EopkP5dHukpOmT3L
2QLzS7wXcelJaUZ5LdzmUk
2dHjmuCuQuToZQ2sSNqPyo
1KOZFJLoViAWnLJwm4AU1Y
1tRlGMHsf21FDo6pja27YN
4XWa9Ul25S9X0QuQs01VAS
4umIPjkehX1r7uhmGvXiSV
4ZsSg2QjbPIYbHDCcKymxu
03ra0YbDqiNBdJh00n5kFB
3S3DDZrPOYTirleXRyHnNM
3K8jhwWeVxwnt7VguiRkZp
3mMfw2dLgxpNNcQzGwtAyw
67cpn8ZBO7QU67jky8Fh47
0DQeHG1cTnvO49JNIYPfIB
10MrjXPa3Fzu0zb2D0muyd
43dQwFD0vtPSprSEgGUceQ
5U6sM9kRxOJ2dXPyUT4w7l
5WvAo7DNuPRmk4APhdPzi8
4OuGgwdH8WCNhcpiF8HoZH
4XOcBYsZrR8oa9c97COoYf
7AAt2NDyAnv

2H7F7EfsVhy0jNLsVz8MLH
3Pf5BhOerMmF17pql51761
7rLegaz7zKB6EtDNzcslKE
6AGOKlMZWLCaEJGnaROtF9
350aUOZhGtTOK1dQbQFKNc
60wF0XnX7A0jVpDNA5v8vi
7f9BjtgQAnsjMfJMHGz2h3
5MwfxCtqMFGYp9Nc1BkTrS
4tYpkgBjxxin7SiUBW0Rtq
7GmBVU53MPqKGJN2LURdTO
5Oh8sPyFYM4gAR3KmmDH7I
3BA7bY2NRzGuijuaQUVXgI
5KbBGUpIZd6gVFUU8tqYiQ
0dk1IMtJj58PMch8ycxfV9
6aNxF6lbcwr8iA8EHOW3Xr
6xtFYAO5wpAjJvi90JTsRd
3VEvsFY82950YaqNz2vvyr
2q34a2BWlNX0kF66STcJkm
0GU5GuJQQEnnREO2FxnVPT
0sPeWsmk1HC2LMN7dnVEeA
2J9TGb5CRT4omfAgnKmXn5
0Yq8benrZdq4onDUPW1DXh
5cCpq6EJqI8dgz8FwE7gk3
2ChZhwJMkwHw6IepF5oGzm
3hbTkJ0yFzHbawnKyhvTQj
4u7C7C7nC7P8mB2LuWxph0
0aZMdAA6pg0IdH1r9MF5sT
6RyuoOJXNzlVWpfC5xQyeI
4punHldq7Jfx0Plu7SdjQ0
6vo7Y1iWyZvpjvFbf6BiZ5
67XyC1GdSkKb9sbRRkUDXS
07c11BxeE28llfHSPSRqFv
4saklk6nie3yiGePpBwUoc
5L1HQLXeih82ecmY0tOH7w
5f2TWu6R2YYCJtLQ0fP78H
0tKl8ZVM6xJkhznvJAS3Nq
6z9MC6IUgZ3lL1cwPbucyX
23UrAQkpPDm0lyokbduSlN
2NbUnaGrjwRuRjZ5EPyOR5
4V71QgMz4NY9TGHYLhwoGA
2r4FPlaVXRdR2nSbUqcTgr
5zsHmE2gO3RefVsPyw2e3T
17VsiwLi1ovKYdVUqDieGC
7kuRvjpeUcn

1wO8SDfVu6xHB1tpprnGja
0HVHwTlOA0piDJYPWUDskL
4Hz4ZddemX2Ij985XRxC2q
40tFJtuES1da2fg0OurUdI
03sEzk1VyrUZSgyhoQR0LZ
7ykaQejTPvmDjf5NLvhTmT
6n4U3TlzUGhdSFbUUhTvLP
3ld409h4DdrdPJzEdeU1WS
1i0aEIsCWwsmN5pn0oatfN
31UjYv5vLsDkzchJOtUMtW
27UemtTEEyH7PYVBnQDGcO
2ZwcnFDvQueNemTt89qCom
1kWbiIJpHqDo1KWS9WpCy6
7COXchtUOMd6uIT6HvmRaI
2U0reegy9weZepp55bazu3
1mjHyr9e7uP0QisX3y7a6T
0ASX6sVGUuOxozyIipE67O
51lR43HOHHF0REvbSSoYU3
27F0lZwmXUe9PxqRqfT78z
4xnZyo0EmLQ17JYHynAGRZ
15RRpHBNzv0gvjIry4jyOs
43067bbX2FhYHpfAntTIYB
0H8CBal2Dcu5Yc001VotZN
5MhZLInNFjr49mX1ICnmzF
0cRgJuB6TRtb5RZ2HkjSLh
1Xi3XkDHp9dTeBpaxFVicJ
7cz506PyIgh2bMN90Mc8Tn
1eSxcMkl6qvqJMT6d6GoCb
0yeeZPK0iDeIDdAfz9yyus
5ZEQjTroORCu6uWvZrdeNc
2OWj7s5FvEHmCoZz5vKY6O
2ksOAxtIxY8yElEWw8RhgK
4Jv7U0JJpbQnOrjtDwDZTZ
4beibTdsFNXHq7bosci34R
1Ik4wlvUOmYwk4S5UZAnzy
5PNlZ3ppftP5nuC9dylWLn
54nChLooSgU5bUQCV2oVOY
4S4mUQoI7yqnnyEAfh7MAP
4b8JWkJ7X8Pdpc3gXBidZ6
0QvJ5pNWlnMiLIxr7M6t3N
2GiJYvgVaD2HtM8GqD9EgQ
6Z4vonpOado2slZz4XFf0P
4Pqa3fbtml8hIMJP246Qtz
3Yoy8R29Iew

In [27]:
print(len(viral_features_df))

# null values left
print(viral_features_df.isnull().sum())
viral_features_df.head()

6008
ID                  0
duration_ms         0
popularity          0
acousticness        0
danceability        0
energy              0
key                 0
instrumentalness    0
liveness            0
loudness            0
mode                0
speechiness         0
valence             0
tempo               0
loud_start          0
dtype: int64


,ID,duration_ms,popularity,acousticness,danceability,energy,key,instrumentalness,liveness,loudness,mode,speechiness,valence,tempo,loud_start
0,5PWzxlxfeowsA6QaNUQhVZ,189610,0,0.209,0.619,0.555,11,0,0.3100,-10.837,0,0.2760,0.438,168.115,-17.812
1,2gXZp1zDDA5bnPsLwrSMRj,303755,63,0.608,0.716,0.626,1,8.26e-06,0.1910,-5.274,1,0.0347,0.467,120.021,-10.877
2,6A9QEWYXpXLMeJLRjGsIJ1,209320,55,0.407,0.442,0.787,8,0,0.1080,-3.889,0,0.0937,0.414,127.979,-10.903
3,5kQre7o8ky5DXeEPcyaCR1,192698,74,0.364,0.565,0.781,10,0,0.0984,-3.358,0,0.0610,0.943,129.959,-3.946
4,6FyRXC8tJUh863JCkyWqtk,258638,71,0.323,0.767,0.709,1,0,0.0676,-4.470,1,0.3360,0.720,171.993,-9.615


# Leftjoin 
left join the 13282 charts entries with the dataframe of the unique 5872 music features

In [28]:
viral_df=viral_df_basic_top20.merge(viral_features_df, how='left',on='ID')


In [29]:
#drop rows with missing music data - most notably Baccara "Yes Sir I can boogey"
viral_df.dropna(axis=0, how='any', thresh=17, inplace=True)
len(viral_df)

13699

# EXPORT TO CSV
Analysis will happen in jupyter notebook 'analysis'

In [30]:
viral_df.rename(columns={'length':'duration_ms'})

,Position,Track Name,Artist,URL,filename,ID,date,country,group,duration_ms,popularity,acousticness,danceability,energy,key,instrumentalness,liveness,loudness,mode,speechiness,valence,tempo,loud_start
0,1,Sucker,Jonas Brothers,https://open.spotify.com/track/4y3OI86AEP6PQoD...,2019-03-27-in-viral.csv,4y3OI86AEP6PQoDE6olYhO,2019-03-27,in,viral,181040,10,0.0427,0.842,0.734,1,0,0.1060,-5.065,0,0.0588,0.952,137.958,-11.142
1,2,Undrunk,FLETCHER,https://open.spotify.com/track/5SHhPFh68OhUmuR...,2019-03-27-in-viral.csv,5SHhPFh68OhUmuRPymKX9d,2019-03-27,in,viral,183428,0,0.4550,0.679,0.572,5,2.41e-06,0.1270,-6.703,1,0.0781,0.542,92.859,-8.190
2,3,Don't Call Me Up,Mabel,https://open.spotify.com/track/5WHTFyqSii0lmT9...,2019-03-27-in-viral.csv,5WHTFyqSii0lmT9R21abT8,2019-03-27,in,viral,178480,79,0.2960,0.674,0.881,9,3.01e-06,0.0793,-2.853,1,0.1470,0.234,98.994,-5.559
3,4,Moonlight,Gaullin,https://open.spotify.com/track/0A5gdlrpAuQqZ2i...,2019-03-27-in-viral.csv,0A5gdlrpAuQqZ2iFgnqBFW,2019-03-27,in,viral,173066,72,0.0529,0.813,0.761,10,0.0119,0.3370,-5.687,0,0.0850,0.290,124.957,-5.704
4,5,i'm so tired...,Lauv,https://open.spotify.com/track/7COXchtUOMd6uIT...,2019-03-27-in-viral.csv,7COXchtUOMd6uIT6HvmRaI,2019-03-27,in,viral,162582,13,0.1760,0.599,0.733,11,0,0.2420,-7.058,1,0.2030,0.534,102.211,-12.880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13696,16,ゴーカ!ごーかい!?ファントムシーフ!,ハロー、ハッピーワールド!,https://open.spotify.com/track/6Mfr5EHIrMhZefe...,2018-04-01-jp-viral.csv,6Mfr5EHIrMhZefeWuOPuiO,2018-04-01,jp,viral,218640,28,0.1580,0.575,0.960,8,0.000104,0.3260,-1.187,0,0.0587,0.719,107.495,-2.885
13697,17,Wager,ichikoro,https://open.spotify.com/track/7kKEqFQaPTmQgW7...,2018-04-01-jp-viral.csv,7kKEqFQaPTmQgW7wadHspJ,2018-04-01,jp,viral,317811,23,0.0270,0.340,0.911,11,0.878,0.0672,-7.353,0,0.0498,0.581,125.282,-19.217
13698,18,見ざる聞かざる言わざる,WEDNESDAY CAMPANELLA,https://open.spotify.com/track/3unHkbItvg3HXYm...,2018-04-01-jp-viral.csv,3unHkbItvg3HXYmIwZVwJW,2018-04-01,jp,viral,232886,4,0.5640,0.689,0.594,6,0.289,0.3180,-10.063,1,0.0607,0.597,135.968,-14.818
13699,19,インターネットじゃカッコいいのに feat. Chiyoko,uno,https://open.spotify.com/track/5KBshrM1lL0a9j1...,2018-04-01-jp-viral.csv,5KBshrM1lL0a9j18Rx7k5S,2018-04-01,jp,viral,262010,3,0.0667,0.741,0.813,9,0.523,0.0256,-5.379,0,0.0455,0.564,120.006,-9.823


In [31]:
viral_df.to_csv(r'data/analysis/viral_charts_songs_2017-2020.csv', index = False)

In [32]:
###########################################